<a href="https://colab.research.google.com/github/KeHang-Zhu/panic_sprea_Depression_1920_1940/blob/main/Heatmap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 55.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable


In [ ]:
data = pd.read_csv('result.txt', sep=',', header=None, names=['state', 'year', 'score', 'number'])
data['panic_level'] = data['score'] / data['number']


In [ ]:
#Calculate the global minimum and maximum panic levels:
min_panic_level = data['panic_level'].min()
max_panic_level = data['panic_level'].max()

In [ ]:
shapefile_path = '/content/ne_50m_admin_1_states_provinces.shp'
usa_map = gpd.read_file(shapefile_path)

# Filter the map to keep only the United States
usa_map = usa_map[usa_map['admin'] == 'United States of America']


In [ ]:
# def create_heatmap(year, data, usa_map, output_folder):
#     data_year = data[data['year'] == year]
#     merged_map = usa_map.set_index('name').join(data_year.set_index('state'))
#     merged_map = merged_map.reset_index()

#     fig, ax = plt.subplots(1, figsize=(20, 12))
#     divider = make_axes_locatable(ax)
#     cax = divider.append_axes("right", size="5%", pad=0.1)
#     merged_map.plot(column='panic_level', cmap='coolwarm_r', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True, cax=cax)

#     ax.set_title(f"Heatmap of Panic Spread Across America's States ({year})", fontsize=20)
#     ax.set_axis_off()

#     plt.savefig(f"{output_folder}/heatmap_{year}.png", dpi=300)
#     plt.close()

def create_heatmap(year, data, usa_map, output_folder, vmin, vmax):
    data_year = data[data['year'] == year]
    merged_map = usa_map.set_index('name').join(data_year.set_index('state'))
    merged_map = merged_map.reset_index()

    fig, ax = plt.subplots(1, figsize=(20, 12))
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.1)
    merged_map.plot(column='panic_level', cmap='coolwarm_r', linewidth=0.8, ax=ax, edgecolor='0.8',
                    legend=True, cax=cax, vmin=vmin, vmax=vmax)

    ax.set_title(f"Heatmap of Panic Spread Across America's States ({year})", fontsize=20)
    ax.set_axis_off()

    plt.savefig(f"{output_folder}/heatmap_{year}.png", dpi=300)
    plt.close()



In [ ]:
output_folder = "heatmaps"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

unique_years = sorted(data['year'].unique())

# for year in unique_years:
#     create_heatmap(year, data, usa_map, output_folder)

for year in unique_years:
    create_heatmap(year, data, usa_map, output_folder, min_panic_level, max_panic_level)


In [ ]:
import cv2
import os

video_name = 'heatmap_evolution.mp4'
frame_rate = 1  # Set the number of frames per second

# Get the dimensions of the first image
first_image = cv2.imread(f"{output_folder}/heatmap_{unique_years[0]}.png")
height, width, _ = first_image.shape

# Set up the video writer with the mp4 codec
video_writer = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v'), frame_rate, (width, height))

# Add each image to the video
for year in unique_years:
    image = cv2.imread(f"{output_folder}/heatmap_{year}.png")
    video_writer.write(image)

# Release the video writer
video_writer.release()

